In [1]:
import pandas as pd
import numpy as np
import time
import gc


In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
dtypes = {
        'ip'            : 'uint64',
        'app'           : 'uint64',
        'device'        : 'uint64',
        'os'            : 'uint64',
        'channel'       : 'uint64',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train = pd.read_csv(path + 'train.csv',dtype=dtypes)
print('train done!')
# test = pd.read_csv(path + 'test.csv',dtype=dtypes)
# print('test done!')
# test_supplement = pd.read_csv(path + 'test_supplement.csv',dtype=dtypes)
# print('supplement done!')


train done!


In [3]:
exp = train[train.is_attributed==1].copy()

In [4]:
exp.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

# Process Timestamp

# add time features

In [5]:
# get timestamp

import multiprocessing

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def get_timestamp(x):
    return x.timestamp()





workers = 30

for df in [exp]:
    clicks = pd.to_datetime(df.click_time)
    print('get clicks')
    df['timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
#     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['timestamp'] = df['timestamp'].astype('uint32')
    print('timestamping is done')
    
    clicks = pd.to_datetime(df.attributed_time)
    print('get clicks')
    df['attributed_timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
#     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['attributed_timestamp'] = df['attributed_timestamp'].astype('uint32')
    print('attributed timestamping is done')

    
    print('================================================================')

get clicks
timestamping is done
get clicks
attributed timestamping is done


In [6]:
exp['timediff'] = exp['attributed_timestamp'] - exp['timestamp']

In [7]:
dff = pd.DataFrame()

dff['timediff'] = exp.groupby('channel').timediff.mean()
dff['timediffmin'] = exp.groupby('channel').timediff.min()
dff['timediffmax'] = exp.groupby('channel').timediff.max()
dff['channel'] = dff.index.values


In [9]:
dff.to_csv('/home/kai/data/kaggle/talkingdata/data/timediff_bychannel.csv', index=False)

In [8]:
dff

,timediff,timediffmin,timediffmax,channel
channel,,,,
0,1721.062992,6,84112,0
3,1458.801105,5,85317,3
4,14386.054054,85,44621,4
5,77.567993,0,23925,5
13,15487.000000,95,44181,13
14,340.000000,340,340,14
15,3893.000000,3893,3893,15
17,7719.738095,16,36355,17
18,5233.344828,1,24487,18
